In [12]:

import subprocess
import logging
import socket
import time
from jnpr.junos import Device
from jnpr.junos.exception import ConnectError
import paramiko


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

class DeviceConnectorClass:
    def __init__(self, hostname, ip, username, password, port=22, timeout=30, retries=3, retry_delay=3):
        self.hostname = hostname
        self.ip = ip
        self.username = username
        self.password = password
        self.port = port
        self.timeout = timeout
        self.retries = retries
        self.retry_delay = retry_delay

    def is_valid_hostname_or_ip(self):
        """
        Checks if the provided hostname or IP address can be resolved.
        """
        try:
            socket.gethostbyname(self.hostname)
            return True
        except socket.error as e:
            logging.error(f"Failed to resolve hostname or IP {self.hostname}: {e}")
            return False

    def connect_to_device(self):
        """
        Establishes a connection to the device with retries.
        """
        logging.info(f"Attempting to connect to device: {self.hostname}")

        if not self.is_valid_hostname_or_ip():
            logging.error(f"Invalid hostname or IP address: {self.hostname}")
            raise ConnectError(host=self.hostname, msg=f"Invalid hostname or IP address: {self.hostname}")

        attempt = 0
        while attempt < self.retries:
            try:
                dev = Device(host=self.hostname, user=self.username, passwd=self.password, port=self.port, timeout=self.timeout)
                dev.open()

                if dev.connected:
                    logging.info(f"Successfully connected to {self.hostname}")
                    return dev  # Return the device object for further use
                else:
                    logging.error(f"Failed to establish connection to {self.hostname}")
                    raise ConnectError(host=self.hostname, msg=f"Failed to connect to {self.hostname}")

            except (ConnectError, paramiko.ssh_exception.SSHException, EOFError) as e:
                logging.error(f"Attempt {attempt + 1} failed for {self.hostname}: {e}")
                attempt += 1
                if attempt < self.retries:
                    logging.info(f"Retrying connection to {self.hostname} after {self.retry_delay} seconds...")
                    time.sleep(self.retry_delay)
                else:
                    logging.error(f"All {self.retries} attempts to connect to {self.hostname} failed.")
                    raise ConnectError(host=self.hostname, msg=str(e))

    def close_connection(self, dev):
        """Closes the device connection if it is open."""
        try:
            if dev and dev.connected:
                dev.close()
                logging.info(f"Closed connection to {self.hostname}")
        except Exception as e:
            logging.error(f"Error closing connection to {self.hostname}: {e}")

def check_link_health(router_details, edges):
    def get_interface_status(device, interface_name):
        try:
            interface_info = device.rpc.get_interface_information(terse=True, interface_name=interface_name)
            interface_status = interface_info.find('.//oper-status').text.lower()
            return interface_status == "up"
        except Exception as e:
            logging.error(f"Error retrieving RPC interface status for {interface_name}: {e}")
            return False

    link_health_status = {}

    def find_device_detail(device_name):
        for rd in router_details:
            if rd['hostname'].startswith(device_name):
                return rd
        logging.warning(f"No match found for device {device_name} in router_details.")
        return None

    for edge in edges:
        source_device = edge['data']['source']
        target_device = edge['data']['target']
        source_interface = edge['data']['id'].split('--')[1]
        target_interface = edge['data']['id'].split('--')[3]

        source_detail = find_device_detail(source_device)
        target_detail = find_device_detail(target_device)

        if not source_detail or not target_detail:
            link_health_status[edge['data']['id']] = 'unknown'
            logging.warning(f"Details missing for source {source_device} or target {target_device}")
            continue

        source_connector = DeviceConnectorClass(
            hostname=source_detail['hostname'],
            ip=source_detail['ip'],
            username=source_detail['username'],
            password=source_detail['password']
        )
        target_connector = DeviceConnectorClass(
            hostname=target_detail['hostname'],
            ip=target_detail['ip'],
            username=target_detail['username'],
            password=target_detail['password']
        )

        try:
            with source_connector.connect_to_device() as source_dev, target_connector.connect_to_device() as target_dev:
                source_status = get_interface_status(source_dev, source_interface)
                target_status = get_interface_status(target_dev, target_interface)

            if source_status and target_status:
                link_health_status[edge['data']['id']] = 'reachable'
                logging.info(f"Link {edge['data']['id']} between {source_device} and {target_device} is reachable.")
            else:
                link_health_status[edge['data']['id']] = 'unreachable'
                logging.info(f"Link {edge['data']['id']} between {source_device} and {target_device} is unreachable.")

        except (ConnectError, paramiko.ssh_exception.SSHException, EOFError) as e:
            logging.error(f"Error checking link {edge['data']['id']} between {source_device} and {target_device}: {str(e)}")
            link_health_status[edge['data']['id']] = 'unreachable'

    return link_health_status


# Test example
edges = [{'data': {'id': 'ny-q5230-04--et-0/0/61--ny-q5230-01--et-0/0/62', 'source': 'ny-q5230-04', 'target': 'ny-q5230-01', 'label': 'et-0/0/61--et-0/0/62'}}, {'data': {'id': 'ny-q5230-04--et-0/0/61--ny-q5230-01--et-0/0/63', 'source': 'ny-q5230-04', 'target': 'ny-q5230-01', 'label': 'et-0/0/61--et-0/0/63'}}, {'data': {'id': 'ny-q5240-13--et-0/0/63:0--ny-q5230-01--et-0/0/60', 'source': 'ny-q5240-13', 'target': 'ny-q5230-01', 'label': 'et-0/0/63:0--et-0/0/60'}}, {'data': {'id': 'ny-q5240-13--et-0/0/63:1--ny-q5230-01--et-0/0/61', 'source': 'ny-q5240-13', 'target': 'ny-q5230-01', 'label': 'et-0/0/63:1--et-0/0/61'}}, {'data': {'id': 'ny-q5240-13--et-0/0/59:0--ny-q5240-q07--et-0/0/63:0', 'source': 'ny-q5240-13', 'target': 'ny-q5240-q07', 'label': 'et-0/0/59:0--et-0/0/63:0'}}, {'data': {'id': 'ny-q5240-13--et-0/0/59:1--ny-q5240-q07--et-0/0/63:1', 'source': 'ny-q5240-13', 'target': 'ny-q5240-q07', 'label': 'et-0/0/59:1--et-0/0/63:1'}}, {'data': {'id': 'ny-q5240-q07--et-0/0/62:0--ny-q5230-01--et-0/0/58', 'source': 'ny-q5240-q07', 'target': 'ny-q5230-01', 'label': 'et-0/0/62:0--et-0/0/58'}}, {'data': {'id': 'ny-q5240-q07--et-0/0/62:1--ny-q5230-01--et-0/0/59', 'source': 'ny-q5240-q07', 'target': 'ny-q5230-01', 'label': 'et-0/0/62:1--et-0/0/59'}}]
router_details= [{'hostname': 'ny-q5230-04.englab.juniper.net', 'ip': 'ny-q5230-04.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5230-01.englab.juniper.net', 'ip': 'ny-q5230-01.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5240-q07.englab.juniper.net', 'ip': 'ny-q5240-q07.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5240-13.englab.juniper.net', 'ip': 'ny-q5240-13.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}]

check_link_health(router_details, edges)


ERROR:root:Failed to resolve hostname or IP ny-q5230-04.englab.juniper.net: [Errno 8] nodename nor servname provided, or not known
ERROR:root:Invalid hostname or IP address: ny-q5230-04.englab.juniper.net


TypeError: __init__() got an unexpected keyword argument 'host'

In [40]:
import logging
import socket
import time
from jnpr.junos import Device
from jnpr.junos.exception import ConnectError
import paramiko

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

class DeviceConnectorClass:
    def __init__(self, hostname, ip, username, password, port=22, timeout=30, retries=1, retry_delay=3):
        self.hostname = hostname
        self.ip = ip
        self.username = username
        self.password = password
        self.port = port
        self.timeout = timeout
        self.retries = retries
        self.retry_delay = retry_delay

    def is_valid_hostname(self):
        """ Checks if the provided hostname can be resolved. """
        try:
            socket.gethostbyname(self.hostname)
            return True
        except socket.error as e:
            logging.error(f"Failed to resolve hostname {self.hostname}: {e}")
            return False

    def connect_to_device(self):
        """ Attempts to connect to the device using hostname first, then IP if hostname fails. """
        for attempt in range(self.retries):
            try:
                # First, attempt to connect using the hostname
                if self.is_valid_hostname():
                    dev = Device(host=self.hostname, user=self.username, passwd=self.password, port=self.port, timeout=self.timeout)
                    dev.open()
                    if dev.connected:
                        logging.info(f"Successfully connected to {self.hostname} using hostname")
                        return dev
                else:
                    logging.warning(f"Hostname {self.hostname} unreachable, attempting connection using IP {self.ip}")

                # If hostname fails, attempt to connect using the IP address directly
                dev = Device(host=self.ip, user=self.username, passwd=self.password, port=self.port, timeout=self.timeout)
                dev.open()
                if dev.connected:
                    logging.info(f"Successfully connected to {self.ip} using IP after hostname failure")
                    return dev

                # If connection fails with both hostname and IP
                logging.error(f"Failed to establish connection to {self.hostname} or IP {self.ip}")
                return None

            except (ConnectError, paramiko.ssh_exception.SSHException, EOFError) as e:
                logging.error(f"Attempt {attempt + 1} failed for {self.hostname} or IP {self.ip}: {e}")
                if attempt < self.retries - 1:
                    logging.info(f"Retrying connection after {self.retry_delay} seconds...")
                    time.sleep(self.retry_delay)
        logging.error(f"All {self.retries} attempts to connect to {self.hostname} or IP {self.ip} failed.")
        return None

    def close_connection(self, dev):
        """Closes the device connection if it is open."""
        try:
            if dev and dev.connected:
                dev.close()
                logging.info(f"Closed connection to {self.hostname} (or IP {self.ip})")
        except Exception as e:
            logging.error(f"Error closing connection to {self.hostname} or IP {self.ip}: {e}")

def check_link_health(router_details, edges):
    def get_interface_status(device, interface_name):
        """ Use RPC to retrieve interface status for a given interface on a connected device. """
        try:
            interface_info = device.rpc.get_interface_information(terse=True, interface_name=interface_name)
            interface_status = interface_info.find('.//oper-status').text.lower()
            return interface_status == "up"
        except Exception as e:
            logging.error(f"Error retrieving RPC interface status for {interface_name}: {e}")
            return False

    # Pre-check reachability for all devices in router_details
    reachable_devices = {}
    for device in router_details:
        connector = DeviceConnectorClass(
            hostname=device['hostname'],
            ip=device['ip'],
            username=device['username'],
            password=device['password']
        )
        dev = connector.connect_to_device()
        if dev:
            reachable_devices[device['hostname']] = device
            connector.close_connection(dev)
        else:
            logging.warning(f"Skipping unreachable device: {device['hostname']}")

    # Skip link processing if no devices are reachable
    if not reachable_devices:
        logging.warning("No reachable devices found. Skipping link processing.")
        return {}

    link_health_status = {}

    def find_device_detail(device_name):
        """ Finds a device in reachable_devices by either matching hostname prefix or IP address. """
        for rd in reachable_devices.values():
            if rd['hostname'].startswith(device_name) or rd['ip'] == device_name:
                return rd
        logging.warning(f"No match found for device {device_name} in reachable devices.")
        return None

    # Process each link only if we have reachable devices
    for edge in edges:
        source_device = edge['data']['source']
        target_device = edge['data']['target']
        source_interface = edge['data']['id'].split('--')[1]
        target_interface = edge['data']['id'].split('--')[3]

        source_detail = find_device_detail(source_device)
        target_detail = find_device_detail(target_device)

        if not source_detail or not target_detail:
            link_health_status[edge['data']['id']] = 'unknown'
            logging.warning(f"Details missing for source {source_device} or target {target_device}")
            continue

        source_connector = DeviceConnectorClass(
            hostname=source_detail['hostname'],
            ip=source_detail['ip'],
            username=source_detail['username'],
            password=source_detail['password']
        )
        target_connector = DeviceConnectorClass(
            hostname=target_detail['hostname'],
            ip=target_detail['ip'],
            username=target_detail['username'],
            password=target_detail['password']
        )

        try:
            # Attempt to connect to both devices
            source_dev = source_connector.connect_to_device()
            target_dev = target_connector.connect_to_device()

            if source_dev is None or target_dev is None:
                link_health_status[edge['data']['id']] = 'unreachable'
                logging.info(f"Link {edge['data']['id']} between {source_device} and {target_device} is unreachable due to connection failure.")
                continue

            # Check interface status on both devices
            source_status = get_interface_status(source_dev, source_interface)
            target_status = get_interface_status(target_dev, target_interface)

            if source_status and target_status:
                link_health_status[edge['data']['id']] = 'reachable'
                logging.info(f"Link {edge['data']['id']} between {source_device} and {target_device} is reachable.")
            else:
                link_health_status[edge['data']['id']] = 'unreachable'
                logging.info(f"Link {edge['data']['id']} between {source_device} and {target_device} is unreachable.")

        except Exception as e:
            logging.error(f"Error checking link {edge['data']['id']} between {source_device} and {target_device}: {str(e)}")
            link_health_status[edge['data']['id']] = 'unreachable'
        
        finally:
            # Ensure connections are closed after each check
            if source_dev:
                source_connector.close_connection(source_dev)
            if target_dev:
                target_connector.close_connection(target_dev)

    return link_health_status

# Test example
edges = [{'data': {'id': 'ny-q5230-00--et-0/0/61--ny-q5230-01--et-0/0/62', 'source': 'ny-q5230-04', 'target': 'ny-q5230-01', 'label': 'et-0/0/61--et-0/0/62'}}, {'data': {'id': 'ny-q5230-04--et-0/0/61--ny-q5230-01--et-0/0/63', 'source': 'ny-q5230-04', 'target': 'ny-q5230-01', 'label': 'et-0/0/61--et-0/0/63'}}, {'data': {'id': 'ny-q5240-13--et-0/0/63:0--ny-q5230-01--et-0/0/60', 'source': 'ny-q5240-13', 'target': 'ny-q5230-01', 'label': 'et-0/0/63:0--et-0/0/60'}}, {'data': {'id': 'ny-q5240-13--et-0/0/63:1--ny-q5230-01--et-0/0/61', 'source': 'ny-q5240-13', 'target': 'ny-q5230-01', 'label': 'et-0/0/63:1--et-0/0/61'}}, {'data': {'id': 'ny-q5240-13--et-0/0/59:0--ny-q5240-q07--et-0/0/63:0', 'source': 'ny-q5240-13', 'target': 'ny-q5240-q07', 'label': 'et-0/0/59:0--et-0/0/63:0'}}, {'data': {'id': 'ny-q5240-13--et-0/0/59:1--ny-q5240-q07--et-0/0/63:1', 'source': 'ny-q5240-13', 'target': 'ny-q5240-q07', 'label': 'et-0/0/59:1--et-0/0/63:1'}}, {'data': {'id': 'ny-q5240-q07--et-0/0/62:0--ny-q5230-01--et-0/0/58', 'source': 'ny-q5240-q07', 'target': 'ny-q5230-01', 'label': 'et-0/0/62:0--et-0/0/58'}}, {'data': {'id': 'ny-q5240-q07--et-0/0/62:1--ny-q5230-01--et-0/0/59', 'source': 'ny-q5240-q07', 'target': 'ny-q5230-01', 'label': 'et-0/0/62:1--et-0/0/59'}}]
router_details= [{'hostname': 'ny-q5230-04.englab.juniper.net', 'ip': 'ny-q5230-04.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5230-01.englab.juniper.net', 'ip': 'ny-q5230-01.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5240-q07.englab.juniper.net', 'ip': 'ny-q5240-q07.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5240-13.englab.juniper.net', 'ip': 'ny-q5240-13.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}]

check_link_health(router_details, edges)


{'ny-q5230-00--et-0/0/61--ny-q5230-01--et-0/0/62': 'reachable',
 'ny-q5230-04--et-0/0/61--ny-q5230-01--et-0/0/63': 'reachable',
 'ny-q5240-13--et-0/0/63:0--ny-q5230-01--et-0/0/60': 'reachable',
 'ny-q5240-13--et-0/0/63:1--ny-q5230-01--et-0/0/61': 'reachable',
 'ny-q5240-13--et-0/0/59:0--ny-q5240-q07--et-0/0/63:0': 'reachable',
 'ny-q5240-13--et-0/0/59:1--ny-q5240-q07--et-0/0/63:1': 'reachable',
 'ny-q5240-q07--et-0/0/62:0--ny-q5230-01--et-0/0/58': 'reachable',
 'ny-q5240-q07--et-0/0/62:1--ny-q5230-01--et-0/0/59': 'reachable'}

In [44]:
import logging
import socket
import subprocess
import time
from jnpr.junos import Device
from jnpr.junos.exception import ConnectError
import paramiko

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

def check_device_health(router_details, devices):
    def is_port_open(hostname_or_ip, port=22):
        """
        Use `nc` (netcat) to check if a port is open on the specified hostname or IP.
        """
        try:
            result = subprocess.run(
                ['nc', '-z', '-w', '3', hostname_or_ip, str(port)],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            return result.returncode == 0  # True if the port is open
        except subprocess.CalledProcessError as e:
            logging.info(f"nc command failed for {hostname_or_ip}:{port}: {e}")
            return False
        except Exception as e:
            logging.error(f"Unexpected error with nc on {hostname_or_ip}:{port}: {e}")
            return False

    health_status = {}
    logging.info(f"router_details: {router_details}")
    logging.info(f"devices: {devices}")

    for device in devices:
        device_id = device.get('id')  # Use 'id' key for device_id
        logging.info(f"Checking device_id: {device_id}")

        # Construct the full hostname to match router_details
        full_hostname = next((d['hostname'] for d in router_details if d['hostname'].startswith(device_id)), None)

        if not full_hostname:
            health_status[device_id] = 'unknown'
            logging.warning(f"No matching details found for device {device_id} in router details.")
            continue

        # Get IP from the device details
        device_detail = next((d for d in router_details if d['hostname'] == full_hostname), None)
        device_ip = device_detail.get('ip')
        reachable = False

        try:
            # First try using hostname
            reachable = is_port_open(full_hostname)
            if not reachable:
                logging.warning(f"Hostname {full_hostname} unreachable, attempting connection using IP {device_ip}")
                # If hostname fails, try using the IP address
                reachable = is_port_open(device_ip)

            health_status[device_id] = 'reachable' if reachable else 'unreachable'
        except Exception as e:
            health_status[device_id] = 'unreachable'
            logging.info(f"Error connecting to {full_hostname} (or IP {device_ip}): {e}")

    return health_status

router_details= [{'hostname': 'ny-q5230-04.englab.juniper.net', 'ip': 'ny-q5230-04.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5230-01.englab.juniper.net', 'ip': 'ny-q5230-01.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5240-q07.englab.juniper.net', 'ip': 'ny-q5240-q07.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}, {'hostname': 'ny-q5240-13.englab.juniper.net', 'ip': 'ny-q5240-13.englab.juniper.net', 'username': 'root', 'password': 'Embe1mpls'}]
devices= [{'id': 'ny-q5230-04', 'label': 'ny-q5230-04'}, {'id': 'ny-q5230-01', 'label': 'ny-q5230-01'}, {'id': 'ny-q5240-13', 'label': 'ny-q5240-13'}, {'id': 'ny-q5240-q07', 'label': 'ny-q5240-q07'}]
device_detail= None


print(check_device_health(router_details, devices))


{'ny-q5230-04': 'reachable', 'ny-q5230-01': 'reachable', 'ny-q5240-13': 'reachable', 'ny-q5240-q07': 'reachable'}


In [52]:
import logging
import re

unique_connections = [
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/58', 'ny-q5240-q07': 'et-0/0/62:0'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/59', 'ny-q5240-q07': 'et-0/0/62:1'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/60', 'ny-q5240-13': 'et-0/0/63:0'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/61', 'ny-q5240-13': 'et-0/0/63:1'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/62', 'ny-q5230-04': 'et-0/0/60'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/63', 'ny-q5230-04': 'et-0/0/61'},
    {'ny-q5230-04.englab.juniper.net': 'et-0/0/60', 'ny-q5230-01': 'et-0/0/62'},
    {'ny-q5230-04.englab.juniper.net': 'et-0/0/61', 'ny-q5230-01': 'et-0/0/63'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/63:0', 'ny-q5230-01': 'et-0/0/60'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/63:1', 'ny-q5230-01': 'et-0/0/61'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/59:0', 'ny-q5240-q07': 'et-0/0/63:0'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/59:1', 'ny-q5240-q07': 'et-0/0/63:1'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/62:0', 'ny-q5230-01': 'et-0/0/58'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/62:1', 'ny-q5230-01': 'et-0/0/59'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/63:0', 'ny-q5240-13': 'et-0/0/59:0'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/63:1', 'ny-q5240-13': 'et-0/0/59:1'}
]

csv_content = "device1,interface1,device2,interface2\n"
skip_device_patterns = ['mgmt', 'management', 'hypercloud']
skip_interfaces = {'re0:mgmt-0', 'em0', 'fxp0'}
interface_pattern = r"^(et|ge|xe|em|re|fxp)\-[0-9]+\/[0-9]+\/[0-9]+(:[0-9]+)?$"
for connection in unique_connections:
    keys = list(connection.keys())
    if len(keys) < 2:
        logging.warning(f"Skipping connection with only one device: {connection}")
        continue
    device1 = keys[0]
    device2 = keys[1]
    interface1 = connection[device1]
    interface2 = connection[device2]

    if interface1 in skip_interfaces or interface2 in skip_interfaces:
        logging.info(
            f"Skipping connection with ignored interfaces: {device1} ({interface1}), {device2} ({interface2})")
        continue

    if any(pattern in device1.lower() for pattern in skip_device_patterns) or \
            any(pattern in device2.lower() for pattern in skip_device_patterns):
        logging.info(f"Skipping connection with ignored device patterns: {device1}, {device2}")
        continue

    if not re.match(interface_pattern, interface1) or not re.match(interface_pattern, interface2):
        logging.warning(
            f"Skipping connection due to invalid interface format: {device1} ({interface1}), {device2} ({interface2})")
        continue

    csv_content += f"{device1},{interface1},{device2},{interface2}\n"

print(csv_content)

device1,interface1,device2,interface2
ny-q5230-01.englab.juniper.net,et-0/0/58,ny-q5240-q07,et-0/0/62:0
ny-q5230-01.englab.juniper.net,et-0/0/59,ny-q5240-q07,et-0/0/62:1
ny-q5230-01.englab.juniper.net,et-0/0/60,ny-q5240-13,et-0/0/63:0
ny-q5230-01.englab.juniper.net,et-0/0/61,ny-q5240-13,et-0/0/63:1
ny-q5230-01.englab.juniper.net,et-0/0/62,ny-q5230-04,et-0/0/60
ny-q5230-01.englab.juniper.net,et-0/0/63,ny-q5230-04,et-0/0/61
ny-q5230-04.englab.juniper.net,et-0/0/60,ny-q5230-01,et-0/0/62
ny-q5230-04.englab.juniper.net,et-0/0/61,ny-q5230-01,et-0/0/63
ny-q5240-13.englab.juniper.net,et-0/0/63:0,ny-q5230-01,et-0/0/60
ny-q5240-13.englab.juniper.net,et-0/0/63:1,ny-q5230-01,et-0/0/61
ny-q5240-13.englab.juniper.net,et-0/0/59:0,ny-q5240-q07,et-0/0/63:0
ny-q5240-13.englab.juniper.net,et-0/0/59:1,ny-q5240-q07,et-0/0/63:1
ny-q5240-q07.englab.juniper.net,et-0/0/62:0,ny-q5230-01,et-0/0/58
ny-q5240-q07.englab.juniper.net,et-0/0/62:1,ny-q5230-01,et-0/0/59
ny-q5240-q07.englab.juniper.net,et-0/0/63:0,ny-q52

In [54]:
import re
import logging

# Sample data
unique_connections = [
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/58', 'ny-q5240-q07': 'et-0/0/62:0'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/59', 'ny-q5240-q07': 'et-0/0/62:1'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/60', 'ny-q5240-13': 'et-0/0/63:0'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/61', 'ny-q5240-13': 'et-0/0/63:1'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/62', 'ny-q5230-04': 'et-0/0/60'},
    {'ny-q5230-01.englab.juniper.net': 'et-0/0/63', 'ny-q5230-04': 'et-0/0/61'},
    {'ny-q5230-04.englab.juniper.net': 'et-0/0/60', 'ny-q5230-01': 'et-0/0/62'},
    {'ny-q5230-04.englab.juniper.net': 'et-0/0/61', 'ny-q5230-01': 'et-0/0/63'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/63:0', 'ny-q5230-01': 'et-0/0/60'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/63:1', 'ny-q5230-01': 'et-0/0/61'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/59:0', 'ny-q5240-q07': 'et-0/0/63:0'},
    {'ny-q5240-13.englab.juniper.net': 'et-0/0/59:1', 'ny-q5240-q07': 'et-0/0/63:1'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/62:0', 'ny-q5230-01': 'et-0/0/58'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/62:1', 'ny-q5230-01': 'et-0/0/59'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/63:0', 'ny-q5240-13': 'et-0/0/59:0'},
    {'ny-q5240-q07.englab.juniper.net': 'et-0/0/63:1', 'ny-q5240-13': 'et-0/0/59:1'}
]

def strip_domain(hostname):
    """Strip the domain from a hostname."""
    return hostname.split('.')[0]

# Set to store unique connections
unique_connections_set = set()

# Process each connection
for connection in unique_connections:
    # Remove domain from hostnames
    stripped_connection = {strip_domain(k): v for k, v in connection.items()}
    # Sort items in the connection and convert to a tuple to ensure uniqueness
    sorted_connection = tuple(sorted(stripped_connection.items()))
    # Add to the set
    unique_connections_set.add(sorted_connection)

# Convert the set back to a list for display or further processing
unique_connections_list = [dict(connection) for connection in unique_connections_set]

# Print the unique connections without domain names
logging.info(f"Unique connections without domains: {unique_connections_list}")
for conn in unique_connections_list:
    print(conn)


{'ny-q5230-01': 'et-0/0/62', 'ny-q5230-04': 'et-0/0/60'}
{'ny-q5240-13': 'et-0/0/59:1', 'ny-q5240-q07': 'et-0/0/63:1'}
{'ny-q5240-13': 'et-0/0/59:0', 'ny-q5240-q07': 'et-0/0/63:0'}
{'ny-q5230-01': 'et-0/0/60', 'ny-q5240-13': 'et-0/0/63:0'}
{'ny-q5230-01': 'et-0/0/61', 'ny-q5240-13': 'et-0/0/63:1'}
{'ny-q5230-01': 'et-0/0/58', 'ny-q5240-q07': 'et-0/0/62:0'}
{'ny-q5230-01': 'et-0/0/59', 'ny-q5240-q07': 'et-0/0/62:1'}
{'ny-q5230-01': 'et-0/0/63', 'ny-q5230-04': 'et-0/0/61'}
